In [1]:
import pandas as pd
from IPython.display import display, clear_output
import time
from datetime import datetime,timedelta
import numpy as np
from scipy.stats import norm

### Connection

In [152]:
from ib_insync import *
util.startLoop()

ibVIX = IB()
ibSPX = IB()
ibVIX.connect('127.0.0.1', 7497, clientId=15)
ibSPX.connect('127.0.0.1', 7497, clientId=16)
ibVIX.client.reqMarketDataType( 3 )
ibSPX.client.reqMarketDataType( 3 )

### Read config

In [153]:
cfg = pd.read_json('config.json').set_index('name')
cfg

,CALL_GAP_ALLOW,HARD_CODE_CUM_PROB,INT_RATE,OBSERVED_CUM_PROB_RECENT,OVERRIDE_FORWARD_DATE,PUT_GAP_ALLOW,USE_OVERRIDE_VVIX,VVIX_OVERNIGHT_HARD_CODE,optionsExp,symbol
name,,,,,,,,,,
second,5,0.98,0.025,0.9760,1.0,5,1.0,89.0,20190521,VXG9
front,5,0.84,0.025,0.8617,1.0,5,1.0,73.5,20190115,VXF9
sp,10,NaN,0.025,NaN,NaN,10,NaN,NaN,20190117,ESH9


### Get SPX futures expirations

In [154]:
def is_third_friday(d):
    return d.weekday() == 4 and 15 <= d.day <= 21

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

In [155]:
ib=ibSPX
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
SPX=pd.DataFrame(sorted(list(chain.expirations)),columns=['expiration'])

#spxFuts = Future(exchange="GLOBEX",symbol = "SPX",tradingClass = 'SP',secType = "FUT")        
#chains = ib.reqContractDetails(spxFuts)
#spxFuts=util.df(chains)
#SPX=pd.DataFrame(data=spxFuts.contract.apply(lambda x:x.lastTradeDateOrContractMonth).values,index=spxFuts.contract.apply(lambda x:x.localSymbol),columns=['expiration'])

SPX['EXPIRATION']=SPX.apply(lambda x:datetime.strptime(str(x.expiration), '%Y%m%d'),axis=1) 
SPX['IsThirdFriday'] = SPX.EXPIRATION.apply(lambda x: is_third_friday(x+ timedelta(days=1)))
SPX.loc[SPX['IsThirdFriday'],'EXPIRATION'] +=  timedelta(days=1)

SPX['FWD_START_TIME'] = SPX['EXPIRATION'] + timedelta(minutes=9.5*60)
SPX['WeekDay'] = SPX.EXPIRATION.apply(lambda x: x.weekday())
SPX['IsLastMonthDay'] = SPX.EXPIRATION.apply(lambda x: last_day_of_month(x)==x)

SPX['IsGoodSpx']=SPX.apply(lambda x:not((x.WeekDay==0) | (x.WeekDay==2)) | x.IsLastMonthDay, axis=1)
SPX=SPX[SPX['IsGoodSpx']]
SPX=SPX.sort_values('EXPIRATION')
SPX['FWD_START_TIME'] = SPX['EXPIRATION'] + timedelta(minutes=16*60+15)
SPX.loc[SPX['IsThirdFriday'],'FWD_START_TIME'] = SPX['EXPIRATION'] + timedelta(minutes=9.5*60)
SPX

,expiration,EXPIRATION,IsThirdFriday,FWD_START_TIME,WeekDay,IsLastMonthDay,IsGoodSpx
0,20190117,2019-01-18,True,2019-01-18 09:30:00,4,False,True
1,20190214,2019-02-15,True,2019-02-15 09:30:00,4,False,True
2,20190314,2019-03-15,True,2019-03-15 09:30:00,4,False,True
3,20190620,2019-06-21,True,2019-06-21 09:30:00,4,False,True
4,20190919,2019-09-20,True,2019-09-20 09:30:00,4,False,True
5,20191219,2019-12-20,True,2019-12-20 09:30:00,4,False,True
6,20200116,2020-01-17,True,2020-01-17 09:30:00,4,False,True
7,20200319,2020-03-20,True,2020-03-20 09:30:00,4,False,True
8,20200618,2020-06-19,True,2020-06-19 09:30:00,4,False,True
9,20201217,2020-12-18,True,2020-12-18 09:30:00,4,False,True


### Get VIX futures and options expirations

In [156]:
ib=ibVIX
vixFut = Future(exchange="CFE",symbol = "VIX",tradingClass = 'VX',secType = "FUT")        
chains = ib.reqContractDetails(vixFut)
vixFut=util.df(chains)
VIXF=pd.DataFrame(data=vixFut.contract.apply(lambda x:x.lastTradeDateOrContractMonth).values,index=vixFut.contract.apply(lambda x:x.localSymbol),columns=['expiration'])
VIXF['EXPIRATION']=VIXF.apply(lambda x:datetime.strptime(str(x.expiration), '%Y%m%d'),axis=1)
VIXF['FWD_START_TIME'] = VIXF['EXPIRATION'] + timedelta(minutes=9.5*60)
VIXF['FWD_END_TIME'] = VIXF['FWD_START_TIME'] + timedelta(days = 30)

display(VIXF)

vix = Index('VIX')
ib.qualifyContracts(vix)
chains = ib.reqSecDefOptParams(vix.symbol, '', vix.secType, vix.conId)
chain = next(c for c in chains if c.tradingClass == 'VIX' and c.exchange == 'SMART')
VIX=pd.DataFrame(sorted(list(chain.expirations)),columns=['expiration'])

VIX['EXPIRATION']=VIX.apply(lambda x:datetime.strptime(str(x.expiration), '%Y%m%d'),axis=1)+ timedelta(days=1)
VIX['FWD_START_TIME'] = VIX['EXPIRATION'] + timedelta(minutes=9.5*60)
VIX['FWD_END_TIME'] = VIX['FWD_START_TIME'] + timedelta(days = 30)
VIX

,expiration,EXPIRATION,FWD_START_TIME,FWD_END_TIME
contract,,,,
VXF9,20190116,2019-01-16,2019-01-16 09:30:00,2019-02-15 09:30:00
VXG9,20190213,2019-02-13,2019-02-13 09:30:00,2019-03-15 09:30:00
VXH9,20190319,2019-03-19,2019-03-19 09:30:00,2019-04-18 09:30:00
VXJ9,20190417,2019-04-17,2019-04-17 09:30:00,2019-05-17 09:30:00
VXK9,20190522,2019-05-22,2019-05-22 09:30:00,2019-06-21 09:30:00
VXM9,20190619,2019-06-19,2019-06-19 09:30:00,2019-07-19 09:30:00
VXN9,20190717,2019-07-17,2019-07-17 09:30:00,2019-08-16 09:30:00
VXQ9,20190821,2019-08-21,2019-08-21 09:30:00,2019-09-20 09:30:00


,expiration,EXPIRATION,FWD_START_TIME,FWD_END_TIME
0,20190115,2019-01-16,2019-01-16 09:30:00,2019-02-15 09:30:00
1,20190212,2019-02-13,2019-02-13 09:30:00,2019-03-15 09:30:00
2,20190318,2019-03-19,2019-03-19 09:30:00,2019-04-18 09:30:00
3,20190319,2019-03-20,2019-03-20 09:30:00,2019-04-19 09:30:00
4,20190416,2019-04-17,2019-04-17 09:30:00,2019-05-17 09:30:00
5,20190521,2019-05-22,2019-05-22 09:30:00,2019-06-21 09:30:00
6,20190618,2019-06-19,2019-06-19 09:30:00,2019-07-19 09:30:00


In [157]:
def getSPXFrontContract(FWD_START_TIME,SPX,OVERRIDE_FORWARD_DATE,ind):
    if ind==1:
        if OVERRIDE_FORWARD_DATE==1:
            FT=SPX.loc[SPX.FWD_START_TIME>FWD_START_TIME,'FWD_START_TIME'].min()
        else:
            FT=SPX.loc[SPX.FWD_START_TIME<FWD_START_TIME,'FWD_START_TIME'].max()
    else:
        FT=SPX.loc[SPX.FWD_START_TIME>FWD_START_TIME,'FWD_START_TIME'].min()
        
    return SPX.loc[SPX.FWD_START_TIME==FT].expiration.values[0]

def getSPXBackContract(FWD_END_TIME,SPX,ind):
    FT=SPX.loc[SPX.FWD_START_TIME==FWD_END_TIME]
    
    if FT.shape[0]>0:
        return FT['expiration'].values[0]
    
    if ind==1:
        FT=SPX.loc[SPX.FWD_START_TIME<FWD_END_TIME,'FWD_START_TIME'].max()
    else:
        FT=SPX.loc[SPX.FWD_START_TIME>FWD_END_TIME,'FWD_START_TIME'].min()
        
    return SPX.loc[SPX.FWD_START_TIME==FT].expiration.values[0]

In [158]:
def getVIXFrontContract(RANK):
    i=0
    if RANK!='front':
        i=0
    elif (VIXF.loc[cfg.loc['front'].symbol].FWD_START_TIME - datetime.now()).days<7:
        i=1
    else:
        i=0
    return VIX.loc[VIX.EXPIRATION >=VIXF.loc[cfg.loc[RANK].symbol].EXPIRATION].expiration.values[i]

def getVIXBackContract(RANK):
    i=0
    if RANK!='front':
        i=0
    elif (VIXF.loc[cfg.loc['front'].symbol].FWD_START_TIME - datetime.now()).days<7:
        i=2
    else:
        i=1
    return VIX.loc[VIX.EXPIRATION >=VIXF.loc[cfg.loc[RANK].symbol].EXPIRATION].expiration.values[i]

### Filling static values

In [166]:
data={}
data['front']={}
data['second']={}
data['vixOptions']={}
data['spxOptions']={}
data['futures']={}
    
data['futures'][cfg.loc['sp'].symbol] =  Future(exchange="GLOBEX",symbol = "ES",tradingClass = 'ES',secType = "FUT",localSymbol = cfg.loc['sp'].symbol)
ib.qualifyContracts(data['futures'][cfg.loc['sp'].symbol])

for c in ['front','second']:
    data[c]['outright']  = cfg.loc[c].symbol
    data[c]['FWD_START_TIME'] = VIXF.loc[data[c]['outright']].FWD_START_TIME
    data[c]['FWD_END_TIME'] = VIXF.loc[data[c]['outright']].FWD_END_TIME
    data[c]['OVERRIDE_FORWARD_DATE']=cfg.loc[c,'OVERRIDE_FORWARD_DATE']
    data[c]['USE_OVERRIDE'] = cfg.loc[c,'USE_OVERRIDE_VVIX']
    data[c]['VVIX_OVERNIGHT_HARD_CODE']= cfg.loc[c,'VVIX_OVERNIGHT_HARD_CODE']
    data[c]['HARD_CODE_CUM_PROB'] = cfg.loc[c,'HARD_CODE_CUM_PROB']
    
    data[c]['SPXFrontContract1'] = getSPXFrontContract(data[c]['FWD_START_TIME'],SPX,data[c]['OVERRIDE_FORWARD_DATE'],1)
    data[c]['SPXFrontContract2'] = getSPXFrontContract(data[c]['FWD_START_TIME'],SPX,data[c]['OVERRIDE_FORWARD_DATE'],2)
    data[c]['SPXBackContract1'] = getSPXBackContract(data[c]['FWD_END_TIME'],SPX,1)
    data[c]['SPXBackContract2'] = getSPXBackContract(data[c]['FWD_END_TIME'],SPX,2)
    
    data[c]['VIXFrontContract'] = getVIXFrontContract(c)
    data[c]['VIXBackContract'] = getVIXBackContract(c)
    
    display(data[c])
#display(data)

{'outright': 'VXF9',
 'FWD_START_TIME': Timestamp('2019-01-16 09:30:00'),
 'FWD_END_TIME': Timestamp('2019-02-15 09:30:00'),
 'OVERRIDE_FORWARD_DATE': 1.0,
 'USE_OVERRIDE': 1.0,
 'VVIX_OVERNIGHT_HARD_CODE': 73.5,
 'HARD_CODE_CUM_PROB': 0.84,
 'SPXFrontContract1': '20190117',
 'SPXFrontContract2': '20190117',
 'SPXBackContract1': '20190214',
 'SPXBackContract2': '20190214',
 'VIXFrontContract': '20190115',
 'VIXBackContract': '20190212'}

{'outright': 'VXG9',
 'FWD_START_TIME': Timestamp('2019-02-13 09:30:00'),
 'FWD_END_TIME': Timestamp('2019-03-15 09:30:00'),
 'OVERRIDE_FORWARD_DATE': 1.0,
 'USE_OVERRIDE': 1.0,
 'VVIX_OVERNIGHT_HARD_CODE': 89.0,
 'HARD_CODE_CUM_PROB': 0.98,
 'SPXFrontContract1': '20190214',
 'SPXFrontContract2': '20190214',
 'SPXBackContract1': '20190314',
 'SPXBackContract2': '20190314',
 'VIXFrontContract': '20190212',
 'VIXBackContract': '20190212'}

In [167]:
#ib.cancelMktData(contract)
#ibSPX.disconnect()

In [168]:
for c in ['front','second']:
    #structure for futures==============================================================
    if not data[c]['outright'] in data['futures']:
        data['futures'][data[c]['outright']] =  Future(exchange="CFE",symbol = "VIX",tradingClass = 'VX',secType = "FUT",localSymbol = data[c]['outright'])
        ibVIX.qualifyContracts(data['futures'][data[c]['outright']])
    
    #structure for options==============================================================
    for optionType in ['vixOptions','spxOptions']:  
        if optionType=='vixOptions':
            dates = set([data[c]['VIXFrontContract'],data[c]['VIXBackContract']])
        else:
            dates = set([data[c]['SPXFrontContract1'],data[c]['SPXFrontContract2'],data[c]['SPXBackContract1'],data[c]['SPXBackContract2']])
        
        for key in dates:
            if not key in data[optionType]:
                print (optionType,key)
                data[optionType][key]={}
                if optionType=='vixOptions':
                    opt = Option(symbol='VIX', exchange='SMART',secType = "OPT",tradingClass='VIX',lastTradeDateOrContractMonth=key)
                    data[optionType][key]['Expiration'] = VIX.loc[VIX.expiration==key].FWD_START_TIME.values[0]
                    expiration = VIX[VIX.expiration==key].EXPIRATION.values[0]
                    forward=VIXF[VIXF.EXPIRATION==expiration].index.values[0]
                    data[optionType][key]['Forward'] = forward
                    if not forward in data['futures']:
                        data['futures'][forward] =  Future(exchange="CFE",symbol = "VIX",tradingClass = 'VX',secType = "FUT",localSymbol = data[optionType][key]['Forward'])
                        ibVIX.qualifyContracts(data['futures'][data[optionType][key]['Forward']])
                    
                    chains = ibVIX.reqContractDetails(opt)
                else:
                    opt = Option(symbol='SPX', exchange='SMART',secType = "OPT",lastTradeDateOrContractMonth=key)
                    data[optionType][key]['Expiration'] =  SPX.loc[SPX.expiration==key].FWD_START_TIME.values[0]
                    
                    data[optionType][key]['Forward'] = cfg.loc['sp'].symbol
                    
                    chains = ibSPX.reqContractDetails(opt)
                    
                options=util.df(chains)
                strikes = sorted(set(options.contract.apply(lambda x:x.strike).values))
            
                data[optionType][key]['Contracts'] = options.contract.values
                data[optionType][key]['Symbols'] = options.contract.apply(lambda x:x.localSymbol).values
                data[optionType][key]['Prices'] = pd.DataFrame(index = strikes,columns=['PUT BID','PUT ASK','CALL BID','CALL ASK'])
data['futPrices'] = pd.DataFrame(index=data['futures'].keys(),columns=['bid','ask','mid'])

vixOptions 20190115
vixOptions 20190212
spxOptions 20190117
spxOptions 20190214
spxOptions 20190314


In [174]:
def onTicker(t):
    if True:
        symbol = t.contract.localSymbol
        #print (symbol)
        if symbol in data['futures']:
            data['futPrices'].loc[symbol,['bid','ask','mid']] = (t.bid, t.ask,0.5*(t.bid+ t.ask))
            return
        
        for optionType in ['vixOptions','spxOptions']:
            for key in data[optionType].keys():
                if symbol in data[optionType][key]['Symbols']:
                    strike = t.contract.strike
                    right =  t.contract.right
                    #print (key,strike,right)
                    if right=='P':
                        data[optionType][key]['Prices'].loc[strike,['PUT BID','PUT ASK']] = (t.bid, t.ask)
                    if right=='C':
                        data[optionType][key]['Prices'].loc[strike,['CALL BID','CALL ASK']] = (t.bid, t.ask)
                    break  
                
def onPrice(tickers):
    for t in tickers:
        onTicker(t)

In [175]:
ibVIX.pendingTickersEvent += onPrice
ibSPX.pendingTickersEvent += onPrice

In [176]:
#subscribe for futures prices
for future in data['futures']:
    ticker = ibVIX.reqMktData(data['futures'][future], '', False, False)

Error 10197, reqId 16: No market data during competing live session, contract: Future(conId=299552802, symbol='ES', lastTradeDateOrContractMonth='20190315', multiplier='50', exchange='GLOBEX', currency='USD', localSymbol='ESH9', tradingClass='ES')


In [177]:
data['futPrices']

,bid,ask,mid
ESH9,NaN,NaN,NaN
VXF9,23.9,23.95,23.925
VXG9,22.7,22.75,22.725


In [178]:
#subscribe for vix options prices
for key in data['vixOptions'].keys():
    print (key)
    for contract in data['vixOptions'][key]['Contracts']:
        ticker = ibVIX.reqMktData(contract, '', False, False)
        time.sleep(1.0/60)

20190115
20190212


Started to throttle requests
Stopped to throttle requests


In [180]:
#data['vixOptions']['20190115']['Prices']

Error 10197, reqId 16: No market data during competing live session, contract: Future(conId=299552802, symbol='ES', lastTradeDateOrContractMonth='20190315', multiplier='50', exchange='GLOBEX', currency='USD', localSymbol='ESH9', tradingClass='ES')


In [181]:
#subscribe for spxOptions prices
for key in data['spxOptions'].keys():
    print (key)
    for contract in data['spxOptions'][key]['Contracts']:
        ticker = ibSPX.reqMktData(contract, '', False, False)
        time.sleep(2.0/60)

20190117
20190214
20190314


In [126]:
#data['spxOptions']['20190314']['Prices']

In [182]:
def getTime2Expiration(CURRENT_TIME,T):
    return (T - CURRENT_TIME).total_seconds()/60/525600

def getKZero(Forward_Price,srtikesList):
    upList=([x for x in srtikesList if x>=Forward_Price])
    downList=([x for x in srtikesList if x<Forward_Price])
    if len(upList)>0:
        RoundUpPX = min(upList)
    else:
        RoundUpPX = np.nan
        print ("Error")

    if len(downList)>0:
        RoundDownPX = max(downList)
    else:
        RoundDownPX = np.nan
        print ("Error")
    RoundIncrement = RoundUpPX - RoundDownPX
    KZero =  int(Forward_Price / RoundIncrement)*RoundIncrement
    return KZero

In [210]:
def getVariance(options,T,Forward_Price,PUT_GAP_ALLOW,CALL_GAP_ALLOW,INT_RATE,KZero): 
    
    vixPuts=options[['PUT BID','PUT ASK']]
    vixPuts=vixPuts.sort_index(ascending=False)
    vixPuts['ValidStrike']=1
    vixPuts.loc[vixPuts.index>KZero,'ValidStrike']=0
    vixPuts.loc[vixPuts['PUT BID']<=0,'ValidStrike']=0
    vixPuts.loc[vixPuts['PUT ASK']<=0,'ValidStrike']=0
    vixPuts.loc[(vixPuts['ValidStrike'].rolling(PUT_GAP_ALLOW).sum()-vixPuts['ValidStrike']==0) & \
                (vixPuts.index!=KZero),'ValidStrike']=0   
    vixPuts=vixPuts.sort_index()
    vixPuts['STRIKE']=vixPuts.index
    
    vixCalls=options[['CALL BID','CALL ASK']]
    vixCalls=vixCalls.sort_index(ascending=True)
    vixCalls['ValidStrike']=1
    vixCalls.loc[vixCalls.index<KZero,'ValidStrike']=0
    vixCalls.loc[vixCalls['CALL BID']<=0,'ValidStrike']=0
    vixCalls.loc[vixCalls['CALL ASK']<=0,'ValidStrike']=0
    vixCalls.loc[(vixCalls['ValidStrike'].rolling(CALL_GAP_ALLOW).sum()-vixCalls['ValidStrike']==0) & \
                 (vixCalls.index!=KZero),'ValidStrike']=0 
    vixCalls['STRIKE']=vixCalls.index

    optionsData = vixPuts.merge(vixCalls, on='STRIKE',how='inner')
    optionsData.index=optionsData.STRIKE
    optionsData=optionsData.sort_index()
    optionsData['ValidStrike'] = (optionsData['ValidStrike_x']+optionsData['ValidStrike_y'])>0
    optionsData['StrikeInterval'] = 0.5*(optionsData.loc[optionsData['ValidStrike'],'STRIKE'].shift(-1)-optionsData.loc[optionsData['ValidStrike'],'STRIKE'].shift(1))
    
    tails=optionsData.loc[optionsData['ValidStrike'],'StrikeInterval'].index
    optionsData.loc[tails[0],'StrikeInterval'] = 0.5*(optionsData.loc[tails[1],'STRIKE'] - optionsData.loc[tails[0],'STRIKE'])
    optionsData.loc[tails[-1],'StrikeInterval'] = 0.5*(optionsData.loc[tails[-1],'STRIKE'] - optionsData.loc[tails[-2],'STRIKE'] )    

    optionsData['BID CONTRIB'] = optionsData['PUT BID']
    optionsData.loc[optionsData.STRIKE>KZero,'BID CONTRIB'] = optionsData['CALL BID']
    optionsData.loc[optionsData.STRIKE==KZero,'BID CONTRIB'] = 0.5*(optionsData['PUT BID']+optionsData['CALL BID'])
    
    optionsData['ASK CONTRIB'] = optionsData['PUT ASK']
    optionsData.loc[optionsData.STRIKE>KZero,'ASK CONTRIB'] = optionsData['CALL ASK']
    optionsData.loc[optionsData.STRIKE==KZero,'ASK CONTRIB'] = 0.5*(optionsData['PUT ASK']+optionsData['CALL ASK'])

    optionsData['BID CONTRIB']=optionsData['BID CONTRIB']*np.exp(INT_RATE*T)*optionsData['StrikeInterval']/pow(optionsData.STRIKE,2)
    optionsData['ASK CONTRIB']=optionsData['ASK CONTRIB']*np.exp(INT_RATE*T)*optionsData['StrikeInterval']/pow(optionsData.STRIKE,2)

    result = 0.5*(2/T*optionsData.loc[optionsData['ValidStrike'],'BID CONTRIB'].sum()*100+2/T*optionsData.loc[optionsData['ValidStrike'],'ASK CONTRIB'].sum()*100)
    
    return result

In [214]:
CURRENT_TIME = datetime.now()

for optionType in ['vixOptions','spxOptions']:
    for key in data[optionType].keys():
        Forward_Price = data['futPrices'].loc[data[optionType][key]['Forward'],'mid']
        if optionType=='vixOptions':
            PUT_GAP_ALLOW = cfg.loc['front'].PUT_GAP_ALLOW
            CALL_GAP_ALLOW= cfg.loc['front'].CALL_GAP_ALLOW
            CALL_GAP_ALLOW= cfg.loc['front'].CALL_GAP_ALLOW
            INT_RATE= cfg.loc['front'].INT_RATE
        else:
            PUT_GAP_ALLOW = cfg.loc['sp'].PUT_GAP_ALLOW
            CALL_GAP_ALLOW= cfg.loc['sp'].CALL_GAP_ALLOW
            CALL_GAP_ALLOW= cfg.loc['sp'].CALL_GAP_ALLOW
            INT_RATE= cfg.loc['sp'].INT_RATE
        
        options = data[optionType][key]['Prices']
        expiration = datetime.utcfromtimestamp(data[optionType][key]['Expiration'].astype('O')/1e9)
        data[optionType][key]['Time2Expiration']=getTime2Expiration(CURRENT_TIME,expiration)
        data[optionType][key]['KZero'] = getKZero(Forward_Price,options.index.values)
            
        data[optionType][key]['Variance'] = getVariance(options,data[optionType][key]['Time2Expiration'],Forward_Price,PUT_GAP_ALLOW,CALL_GAP_ALLOW,INT_RATE,data[optionType][key]['KZero'])

In [246]:
print (c)
data[c]['VIXFrontContract']

second


'20190212'

In [247]:
for c in ['front','second']: 
    FORWARD_START_TIME = data['vixOptions'][data[c]['VIXFrontContract']]['Time2Expiration']
    FORWARD_END_TIME = data['vixOptions'][data[c]['VIXBackContract']]['Time2Expiration']
    
    if data[c]['SPXFrontContract1']==data[c]['SPXFrontContract2']: 
        FRONT_FORWARD_INTERP_VARIANCE = data['spxOptions'][data[c]['SPXFrontContract1']]['Variance'] 
    else:    
        FRONT_FORWARD1_TIME = data['spxOptions'][data[c]['SPXFrontContract1']]['Time2Expiration'] 
        FRONT_FORWARD2_TIME = data['spxOptions'][data[c]['SPXFrontContract2']]['Time2Expiration']
        
        FRONT_FORWARD1_VARIANCE = data['spxOptions'][data[c]['SPXFrontContract1']]['Variance']
        FRONT_FORWARD2_VARIANCE = data['spxOptions'][data[c]['SPXFrontContract2']]['Variance']
        
        FRONT_FORWARD_INTERP_VARIANCE = (FRONT_FORWARD1_TIME*FRONT_FORWARD1_VARIANCE+((FRONT_FORWARD2_TIME*FRONT_FORWARD2_VARIANCE-FRONT_FORWARD1_TIME*FRONT_FORWARD1_VARIANCE)/(FRONT_FORWARD2_TIME-FRONT_FORWARD1_TIME))*(FORWARD_START_TIME-FRONT_FORWARD1_TIME))/FORWARD_START_TIME
    
    if data[c]['SPXBackContract1']==data[c]['SPXBackContract2']:
        BACK_FORWARD_INTERP_VARIANCE = data['spxOptions'][data[c]['SPXBackContract1']]['Variance']  
    else:
        BACK_FORWARD1_TIME = data['spxOptions'][data[c]['SPXBackContract1']]['Time2Expiration'] 
        BACK_FORWARD2_TIME = data['spxOptions'][data[c]['SPXBackContract2']]['Time2Expiration']
        
        BACK_FORWARD1_VARIANCE = data['spxOptions'][data[c]['SPXBackContract1']]['Variance']  
        BACK_FORWARD2_VARIANCE = data['spxOptions'][data[c]['SPXBackContract2']]['Variance']
        
        BACK_FORWARD_INTERP_VARIANCE = (BACK_FORWARD1_TIME*BACK_FORWARD1_VARIANCE+((BACK_FORWARD2_TIME*BACK_FORWARD2_VARIANCE-BACK_FORWARD1_TIME*BACK_FORWARD1_VARIANCE)/(BACK_FORWARD2_TIME-BACK_FORWARD1_TIME))*(FORWARD_END_TIME-BACK_FORWARD1_TIME))/FORWARD_END_TIME
        
    FORWARD_VOLATILITY = np.sqrt(((BACK_FORWARD_INTERP_VARIANCE*FORWARD_END_TIME-FRONT_FORWARD_INTERP_VARIANCE*FORWARD_START_TIME)/(FORWARD_END_TIME-FORWARD_START_TIME))/100)*100
    
    FRONT_FORWARD_PRICE = data['futPrices'].loc[data['vixOptions'][data[c]['VIXFrontContract']]['Forward'],'mid']
    BACK_FORWARD_PRICE = data['futPrices'].loc[data['vixOptions'][data[c]['VIXBackContract']]['Forward'],'mid']
    
    KZERO_FRONT = data['vixOptions'][data[c]['VIXFrontContract']]['KZero']
    KZERO_BACK =  data['vixOptions'][data[c]['VIXBackContract']]['KZero']
    
    FRONT_TIME = data['vixOptions'][data[c]['VIXFrontContract']]['Time2Expiration']
    BACK_TIME =  data['vixOptions'][data[c]['VIXBackContract']]['Time2Expiration']
    FRONT_TIME_HOURS = FRONT_TIME*525600
    BACK_TIME_HOURS = BACK_TIME*525600
    
    FRONT_VARIANCE = data['vixOptions'][data[c]['VIXFrontContract']]['Variance']
    BACK_VARIANCE = data['vixOptions'][data[c]['VIXBackContract']]['Variance']
    
    FRONT_ADJUSTED_VARIANCE = FRONT_VARIANCE/100-1/FRONT_TIME*pow(FRONT_FORWARD_PRICE/KZERO_FRONT-1,2)
    BACK_ADJUSTED_VARIANCE = BACK_VARIANCE/100-1/BACK_TIME*pow(BACK_FORWARD_PRICE/KZERO_BACK-1,2)

    FRONT_WEIGHT = 1 if data[c]['VIXFrontContract']==data[c]['VIXBackContract'] else (BACK_TIME_HOURS-43200)/(BACK_TIME_HOURS-FRONT_TIME_HOURS)
    BACK_WEIGHT = 1  if data[c]['VIXFrontContract']==data[c]['VIXBackContract'] else (43200 - FRONT_TIME_HOURS)/(BACK_TIME_HOURS-FRONT_TIME_HOURS)

    VAR = FRONT_VARIANCE if data[c]['VIXFrontContract']==data[c]['VIXBackContract'] else ((FRONT_TIME*FRONT_ADJUSTED_VARIANCE*FRONT_WEIGHT)+(BACK_TIME*BACK_ADJUSTED_VARIANCE*BACK_WEIGHT))*1200

    VVIX_ADJUSTMENT = FORWARD_VOLATILITY*((1-np.sqrt(1-(data[c]['VVIX_OVERNIGHT_HARD_CODE']/100*FORWARD_START_TIME))) if data[c]['USE_OVERRIDE']==1 else (1-np.sqrt(1-(VAR/100*FORWARD_START_TIME))))

    OUTRIGHT_MIDPOINT =  data['futPrices'].loc[data[c]['outright'],'mid']

    IMPLIED_CUMULATIVE_PROBABILITY = min([norm.cdf((FORWARD_VOLATILITY-OUTRIGHT_MIDPOINT)/VVIX_ADJUSTMENT),0.999999])

    CUMULATIVE_PROBABILITY_USED = data[c]['HARD_CODE_CUM_PROB'] if data[c]['USE_OVERRIDE'] else OBSERVED_CUM_PROB_RECENT

    TV=FORWARD_VOLATILITY+VVIX_ADJUSTMENT*norm.isf(CUMULATIVE_PROBABILITY_USED)
    
    print (TV)
    
    break

24.368025039277427
